# Print length of Temperature

In [10]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

csv_files = glob.glob('./csv_ave/*.csv')  # Assuming CSVs are stored in 'data/' folder

X, Y = [], []  # Lists to store input and output data
df_all = pd.read_csv("all_c6.csv")
n = 0

for file in csv_files:
    df = pd.read_csv(file)  # Load temperature data file
    
    # Extract temperature sequence (all rows)
    avg_temp_sequence = df["ave"] # (time_steps,)


    Y.append(avg_temp_sequence.size)  # Shape: (time_steps,)


print(Y)

[43664, 42816, 22098, 10709, 28965, 35368, 34455, 15497, 10666, 15648, 22232, 28338]


# Test GPT method for different lenght of X

In [1]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load all CSV files
csv_files = glob.glob('./csv_ave/*.csv')  

X, Y = [], []  
df_all = pd.read_csv("all_c6_new.csv")  

n = 0
for file in csv_files:
    df = pd.read_csv(file)  

    # Extract metadata
    initial_speed = df_all["v_set\n[km/h]"][n]
    initial_pressure = df_all["Fb_set\n[kN]"][n]
    braking_time = df_all["braking_time"][n]  # Braking time is the number of rows in the file

    # Extract temperature sequence
    avg_temp_sequence = df["ave"].values  

    # Normalize inputs
    initial_speed /= 100  
    initial_pressure /= 23  
    braking_time /= max(df_all["braking_time"])  # Normalize braking time

    # Store inputs
    X.append([initial_speed, initial_pressure, braking_time])  # Now 3 features
    Y.append(avg_temp_sequence)  

    n += 1

# **Pad sequences to the maximum length**
max_seq_length = max(len(seq) for seq in Y)
Y_padded = pad_sequences(Y, maxlen=max_seq_length, dtype='float32', padding='post', value=0.0)

# Convert X to numpy array
X = np.array(X)  # Shape: (num_samples, 3)

# Reshape X for LSTM
X = X.reshape((X.shape[0], 1, X.shape[1]))  # Shape: (num_samples, 1, 3)
# Repeat X along the time dimension to match Y_padded
X = np.repeat(X, max_seq_length, axis=1)  # Shape: (num_samples, max_seq_length, 3)
# Reshape Y for LSTM
Y_padded = Y_padded.reshape((Y_padded.shape[0], Y_padded.shape[1], 1))  # Shape: (num_samples, max_seq_length, 1)



### **Define LSTM Model**

model = Sequential([
    Masking(mask_value=0.0, input_shape=(max_seq_length, 1)),  # Ignore padding values
    LSTM(64, return_sequences=True),
    LSTM(32, return_sequences=True),
    TimeDistributed(Dense(1))  # Output sequence shape (batch, time_steps, 1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the model
model.fit(X, Y_padded, epochs=50, batch_size=8, validation_split=0.2)

/Users/yanjun/Documents/apps/miniconda3/envs/dd2421/lib/python3.10/site-packages/keras/src/layers/core/masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 43664, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 43664, 64)      │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 43664, 32)      │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 43664, 1)       │            33 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,345 (114.63 KB)

 Trainable params: 29,345 (114.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 3 and 1 for '{{node sequential_1/lstm_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_1/lstm_1/strided_slice_2, sequential_1/lstm_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [?,3], [1,256].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(None, 3), dtype=float32)
  • states=('tf.Tensor(shape=(None, 64), dtype=float32)', 'tf.Tensor(shape=(None, 64), dtype=float32)')
  • training=True

# For same length

In [9]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

csv_files = glob.glob('./csv_ave/*.csv')  # Assuming CSVs are stored in 'data/' folder

X, Y = [], []  # Lists to store input and output data
df_all = pd.read_csv("all_c6.csv")
n = 0

for file in csv_files:
    df = pd.read_csv(file)  # Load temperature data file
    
    # Extract metadata from df_all
    initial_speed = df_all["v_set\n[km/h]"][n]
    initial_pressure = df_all["Fb_set\n[kN]"][n]
    
    # Extract temperature sequence (all rows)
    avg_temp_sequence = df["ave"] # (time_steps,)

    # Normalize inputs
    initial_speed /= 100  # Assuming max speed ~100 km/h
    initial_pressure /= 23  # Assuming max pressure ~23 kN

    # Store in lists
    X.append([initial_speed, initial_pressure])  # Shape: (2,)
    Y.append(avg_temp_sequence)  # Shape: (time_steps,)

    n = n+ 1
    #print(n)

# Convert to numpy arrays
X = np.array(X)  # Shape: (num_tests, 2)
Y = np.array(Y)  # Shape: (num_tests, time_steps)

# Reshape X for LSTM (samples, timesteps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))  # Shape: (num_tests, 1, 2)

# Reshape Y for LSTM (samples, timesteps, features)
Y = Y.reshape((Y.shape[0], Y.shape[1], 1))  # Shape: (num_tests, time_steps, 1)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (12,) + inhomogeneous part.

In [2]:
print(n)

0


In [ ]:
# Define LSTM model
model = Sequential([
    Dense(16, activation="relu", input_shape=(1, 2)),  # Input: speed, pressure
    RepeatVector(Y.shape[1]),  # Repeat input vector for each time step
    LSTM(64, return_sequences=True),  # LSTM learns temperature pattern
    TimeDistributed(Dense(32, activation="relu")),  # Dense applied at each time step
    TimeDistributed(Dense(1))  # Output: temperature sequence
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Train model
model.fit(X, Y, epochs=20, batch_size=4, validation_split=0.2)

# Save model for future predictions
model.save("brake_temp_lstm.h5")


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load trained model
model = load_model("brake_temp_lstm.h5")

# Example input (Speed = 120 km/h, Pressure = 60 bar)
new_input = np.array([[120 / 200, 60 / 100]])  # Normalize values
new_input = new_input.reshape((1, 1, 2))  # Reshape for LSTM

# Predict temperature sequence
predicted_temp_sequence = model.predict(new_input)

# Print results
print(predicted_temp_sequence[0].flatten())  # Output: Temperature over time
